# Prediction of Premier League Results

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
pd.set_option('max_columns', None)

In [2]:
# read and join all teams' stats from last 4 seasons

seasons = ['1718','1819','1920','2021']
full_teamdata = []
best_player = pd.read_csv('poty.csv')
for season in seasons:
    table = pd.read_csv(season + 'table.csv')
    forteam = pd.read_csv(season + 'for.csv').drop(['npxG+xA','xG','xA'],axis=1) #drop some repetitive columns
    against = pd.read_csv(season + 'against.csv')
    shot = pd.read_csv(season + 'shot.csv')
    season_full = pd.concat([table,forteam,against,shot],axis=1)
    season_correct = "20" + season[:2] + "-" + season[2:]
    #Add season number to match with matches dataset
    season_full['league'] = season_correct
    # Add fan voted best player for every team
    season_full['Best Player'] = best_player.dropna(subset=[season_correct]).reset_index()[season_correct]
    full_teamdata.append(season_full)
full_teamtable = pd.concat(full_teamdata,ignore_index=True)

In [3]:
# clean teams' stats

full_teamtable = full_teamtable.drop("MP", axis=1)
full_teamtable = full_teamtable.drop("Attendance", axis=1)

top_scorer = full_teamtable['Top Team Scorer'].str.split(' - ', 1, expand=True)
full_teamtable['Top Scorer'] = top_scorer[0]
full_teamtable['Top Scorer Goals'] = pd.to_numeric(top_scorer[1])
full_teamtable = full_teamtable.drop('Top Team Scorer', axis=1)
full_teamtable = full_teamtable.rename(columns={"Squad":"team"})

full_teamtable

,Rk,team,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,league,Best Player,Top Scorer,Top Scorer Goals
0,6,Arsenal,19,6,13,74,51,23,63,68.4,50.6,17.8,0.47,Petr Čech,30,26.8,61.5,73,61,69,4,5,1.92,1.61,3.53,1.82,3.42,64.6,1.80,1.35,3.15,1.70,3.05,51,38,46,5,6,52,4,1.34,1.00,2.34,1.21,2.21,50.6,46.1,34.0,80.1,1.33,0.89,2.23,1.21,2.11,590,227,38.5,15.53,5.97,0.12,0.30,17.4,30,0.11,4.6,4.4,2017-18,Aaron Ramsey,Alexandre Lacazette,14
1,12,Bournemouth,11,11,16,45,61,-16,44,44.8,61.4,-16.6,-0.44,Asmir Begović,22,26.9,47.8,45,36,42,3,3,1.18,0.95,2.13,1.11,2.05,42.6,1.18,0.85,2.03,1.12,1.97,60,44,58,2,3,73,1,1.58,1.16,2.74,1.53,2.68,61.4,59.3,43.7,103.0,1.62,1.15,2.77,1.56,2.71,473,153,32.3,12.45,4.03,0.09,0.27,17.5,13,0.09,0.2,-0.6,2017-18,Nathan Ake,Callum Wilson,8
2,15,Brighton,9,13,16,34,54,-20,40,37.2,52.6,-15.5,-0.41,Mathew Ryan,24,27.5,43.7,33,25,28,5,7,0.87,0.66,1.53,0.74,1.39,31.8,0.98,0.60,1.58,0.84,1.44,49,36,48,1,3,30,2,1.29,0.95,2.24,1.26,2.21,52.6,50.3,34.5,84.9,1.38,0.91,2.29,1.32,2.23,381,113,29.7,10.03,2.97,0.07,0.25,17.9,3,0.09,-4.2,-3.8,2017-18,Pascal Groß,Glenn Murray,12
3,7,Burnley,14,12,12,36,39,-3,54,34.5,51.7,-17.3,-0.45,Nick Pope,24,27.5,43.9,35,26,35,0,0,0.92,0.68,1.61,0.92,1.61,34.5,0.91,0.63,1.53,0.91,1.53,38,29,35,3,5,45,6,1.00,0.76,1.76,0.92,1.68,51.7,48.2,37.4,85.7,1.36,0.99,2.35,1.27,2.25,387,126,32.6,10.18,3.32,0.09,0.28,16.0,16,0.09,0.5,0.5,2017-18,Nick Pope,Chris Wood,10
4,5,Chelsea,21,7,10,62,38,24,70,57.9,35.3,22.7,0.60,Thibaut Courtois,26,26.7,55.4,60,42,57,3,3,1.58,1.11,2.68,1.50,2.61,55.7,1.52,1.11,2.64,1.46,2.58,36,26,34,2,2,75,1,0.95,0.68,1.63,0.89,1.58,35.3,33.8,25.4,59.1,0.93,0.67,1.60,0.89,1.56,602,209,34.7,15.84,5.50,0.09,0.27,17.4,19,0.09,2.1,1.3,2017-18,N'Golo Kanté,Eden Hazard,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,12,Newcastle Utd,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,Karl Darlow,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,2020-21,Callum Wilson,Callum Wilson,12
64,15,Southampton,12,7,19,47,68,-21,43,42.4,54.2,-11.8,-0.31,Alex McCarthy,29,26.6,52.2,47,33,42,5,6,1.24,0.87,2.11,1.11,1.97,37.9,1.12,0.72,1.84,1.00,1.72,67,44,59,8,9,58,3,1.76,1.16,2.92,1.55,2.71,54.2,47.4,36.3,83.7,1.43,0.95,2.38,1.25,2.20,417,157,37.6,10.97,4.13,0.10,0.27,17.6,20,0.09,4.6,4.1,2020-21,James Ward-Prowse,Danny Ings,12
65,7,Tottenham,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,Hugo Lloris,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,2020-21,Harry Kane,Harry Kane,23
66,6,West Ham,19,8,11,62,47,15,65,53.9,48.3,5.6,0.15,Łukasz Fabiański,24,27.8,42.9,60,46,58,2,4,1.58,1.21,2.79,1.53,2.74,50.7,1.42,0.97,2.39,1.33,2.30,43,34,40,3,5,52,1,1.13,0.89,2.03,1.05,1.95,48.3,44.5,35.3,79.8,1.27,0.93,2.20,1.17,2.10,462,158,34.2,12.16,4.16,0.13,0.37,15.4,16,0.11,6.1,7.3,2020-21,Tomáš Souček,Tomáš Souček,10


In [4]:
# read and clean matches' stats
matches = pd.read_csv('matches.csv')
promoted1819 = ["Wolves","Cardiff City","Fulham"]
promoted1920 = ["Norwich City","Sheffield Utd","Aston Villa"]
promoted2021 = ["Leeds United", "West Bromwich Albion","Fulham"]
promoted2122 = ["Norwich City","Watford","Brentford"]
#match season names to team table names
matches.loc[matches.league == "2018/2019 English Premier League","league"] = "2017-18"
matches.loc[matches.league == "2019-20 English Premier League","league"] = "2018-19"
matches.loc[matches.league == "2020-21 English Premier League","league"] = "2019-20"
matches.loc[matches.league == "2021-22 English Premier League","league"] = "2020-21"
#match team names to team table names
matches.loc[matches.home == "Brighton & Hove Albion","home"] = "Brighton"
matches.loc[matches.away == "Brighton & Hove Albion","away"] = "Brighton"
matches.loc[matches.home == "Huddersfield Town","home"] = "Huddersfield"
matches.loc[matches.away == "Huddersfield Town","away"] = "Huddersfield"
matches.loc[matches.home == "AFC Bournemouth","home"] = "Bournemouth"
matches.loc[matches.away == "AFC Bournemouth","away"] = "Bournemouth"
matches.loc[matches.home == "Wolverhampton Wanderers","home"] = "Wolves"
matches.loc[matches.away == "Wolverhampton Wanderers","away"] = "Wolves"
matches.loc[matches.home == "Sheffield United","home"] = "Sheffield Utd"
matches.loc[matches.away == "Sheffield United","away"] = "Sheffield Utd"
matches.loc[matches.home == "Newcastle United","home"] = "Newcastle Utd"
matches.loc[matches.away == "Newcastle United","away"] = "Newcastle Utd"
matches.loc[matches.home == "Manchester United","home"] = "Manchester Utd"
matches.loc[matches.away == "Manchester United","away"] = "Manchester Utd"
matches.loc[matches.home == "West Ham United","home"] = "West Ham"
matches.loc[matches.away == "West Ham United","away"] = "West Ham"
matches.loc[matches.home == "Tottenham Hotspur","home"] = "Tottenham"
matches.loc[matches.away == "Tottenham Hotspur","away"] = "Tottenham"

In [5]:
# Choose last 4 seasons
matches = matches[matches["league"].isin(["2017-18", "2018-19","2019-20","2020-21"])]
matches.reset_index(inplace=True)
seasons = ['1819','1920','2021','2122']
# Add odds from odds.csv files
all_odds = []
for season in seasons:
    odds = pd.read_csv(season +'odds.csv').loc[:,['AvgH','AvgD','AvgA','Avg>2.5','Avg<2.5']]
    all_odds.append(odds)
all_odds = pd.concat(all_odds,ignore_index=True)
matches = pd.concat([matches,all_odds],axis=1)
# Remove newly promoted teams, because previous season stats from them can't be accessed
matches = matches[((matches["league"] == "2017-18") & (~matches["home"].isin(promoted1819)) & (~matches["away"].isin(promoted1819))) | ((matches["league"] == "2018-19") & (~matches["home"].isin(promoted1920)) & (~matches["away"].isin(promoted1920))) | ((matches["league"] == "2019-20") & (~matches["home"].isin(promoted2021)) & (~matches["away"].isin(promoted2021))) | ((matches["league"] == "2020-21") & (~matches["home"].isin(promoted2122)) & (~matches["away"].isin(promoted2122)))]
matches = matches.fillna(0)
matches = matches.rename(columns={"time (utc)":"time"})

#Add date columns to get a datetime column
month_and_day = matches['date'].apply(lambda date: date.split(", ")[1]).str.split(' ', 1, expand=True)
matches['day'] = month_and_day[1]
matches['day'] = pd.to_numeric(matches['day'])
matches['month'] = month_and_day[0].apply(lambda month: int(datetime.strptime(month, '%B').strftime('%m')))
matches = matches.drop('date', axis=1)

display(matches)

,index,home,away,year,time,attendance,league,home_score,away_score,home_starting_1,home_starting_2,home_starting_3,home_starting_4,home_starting_5,home_starting_6,home_starting_7,home_starting_8,home_starting_9,home_starting_10,home_starting_11,away_starting_1,away_starting_2,away_starting_3,away_starting_4,away_starting_5,away_starting_6,away_starting_7,away_starting_8,away_starting_9,away_starting_10,away_starting_11,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,day,month
0,6462,Manchester Utd,Leicester City,2018,19:00,74439.0,2017-18,2,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8
1,6463,Newcastle Utd,Tottenham,2018,11:30,51749.0,2017-18,1,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,11,8
4,6466,Huddersfield,Chelsea,2018,14:00,24121.0,2017-18,0,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,11,8
5,6467,Watford,Brighton,2018,14:00,20051.0,2017-18,2,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,11,8
7,6469,Liverpool,West Ham,2018,12:30,53235.0,2017-18,4,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,12,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,7714,Burnley,Crystal Palace,2021,15:00,18028.0,2020-21,3,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,20,11
1256,7718,Wolves,West Ham,2021,15:00,30667.0,2020-21,1,0,José Sá,Conor Coady,Romain Saïss,Max Kilman,João Moutinho,Rúben Neves,Rayan Aït-Nouri,Nélson Semedo,Raúl Jiménez,Hwang Hee-Chan,Daniel Podence,Lukasz Fabianski,Kurt Zouma,Craig Dawson,Aaron Cresswell,Ben Johnson,Saïd Benrahma,Declan Rice,Tomás Soucek,Michail Antonio,Pablo Fornals,Jarrod Bowen,3.02,3.27,2.47,2.06,1.79,20,11
1257,7719,Liverpool,Arsenal,2021,17:30,53092.0,2020-21,4,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberlain,Diogo Jota,Sadio Mané,Mohamed Salah,Aaron Ramsdale,Gabriel,Ben White,Nuno Tavares,Takehiro Tomiyasu,Albert Sambi Lokonga,Thomas Partey,Emile Smith Rowe,Bukayo Saka,Pierre-Emerick Aubameyang,Alexandre Lacazette,1.48,4.77,6.38,1.49,2.64,20,11
1258,7720,Manchester City,Everton,2021,14:00,52571.0,2020-21,3,0,Ederson,Aymeric Laporte,John Stones,João Cancelo,Kyle Walker,Rodri,Ilkay Gündogan,Bernardo Silva,Cole Palmer,Phil Foden,Raheem Sterling,Jordan P

In [6]:
#function which is used to convert the whole dataframe to datetime
def toDatetime(x):
    year = str(x.year)
    time = str(x.time)
    month = str(x.month) if x.month > 9 else "0"+str(x.month)
    day = str(x.day) if x.day > 9 else "0"+str(x.day)
    return datetime.fromisoformat(year+"-"+month+"-"+day+"T"+time)

matches["datetime"] = matches[["time","year","day","month"]].apply(toDatetime, axis=1)
matches.drop(["time","year","day","month"], axis=1)
matches.sort_values("datetime", ascending=True,inplace=True)

In [7]:
#Function about last n games goals and points
#returns an array with the structure: [wins, losses, ties, team_goals, opp_goals]
def FindLastGames(n, team, season, datetime):
    results = matches[((matches['home'] == team) | (matches['away'] == team)) & (matches['league'] == season) & (matches['datetime'] < datetime)]
    results.sort_values("datetime", ascending=True,inplace=True)
    
    #if the results are
    if len(results)<n:
        n = len(results)
    
    results = results.tail(n)
    
    #structure - [wins, losses, ties, team_goals, opp_goals]
    out = [0,0,0,0,0]
    
    for i in range(n):
        if results.iloc[i]["away"]==team:
            team = results.iloc[i]["away_score"]
            opponent = results.iloc[i]["home_score"]
        else:
            team = results.iloc[i]["home_score"]
            opponent = results.iloc[i]["away_score"]
        #np.insert(out, 3, team)
        #np.insert(out, 4, opponent)
        out[3]+=team
        out[4]+=opponent
        
        if(team>opponent):
            out[0]+=1
            np.insert(out, 3, team)
        elif(team==opponent):
            out[2]+=1
        else:
            out[1]+=1
    
    return out
#Add columns using the function to get stats from last 5 and 10 games
team_last5 = matches.apply(lambda x: FindLastGames(5, x['home'], x['league'], x['datetime']), axis=1, result_type='expand')
team_last5 = team_last5.rename(columns={0: "wins_in_last5", 1: "losses_in_last5", 2: "ties_in_last5",
                                         3: "goals_in_last5", 4: "goals_against_in_last5"})
matches[["wins_in_last5", "losses_in_last5", "ties_in_last5", "goals_in_last5",
        "goals_against_in_last5"]] = team_last5.astype(int)

opp_last5 = matches.apply(lambda x: FindLastGames(5, x['away'], x['league'], x['datetime']), axis=1, result_type='expand')
opp_last5 = opp_last5.rename(columns={0: "opp_wins_in_last5", 1: "opp_losses_in_last5", 2: "opp_ties_in_last5",
                                         3: "opp_goals_in_last5", 4: "opp_goals_against_in_last5"})
matches[["opp_wins_in_last5", "opp_losses_in_last5", "opp_ties_in_last5", "opp_goals_in_last5",
        "opp_goals_against_in_last5"]] = opp_last5.astype(int)

team_last10 = matches.apply(lambda x: FindLastGames(10, x['home'], x['league'], x['datetime']), axis=1, result_type='expand')
team_last10 = team_last10.rename(columns={0: "wins_in_last10", 1: "losses_in_last10", 2: "ties_in_last10",
                                         3: "goals_in_last10", 4: "goals_against_in_last10"})
matches[["wins_in_last10", "losses_in_last10", "ties_in_last10", "goals_in_last10",
        "goals_against_in_last10"]] = team_last10.astype(int)

opp_last10 = matches.apply(lambda x: FindLastGames(10, x['away'], x['league'], x['datetime']), axis=1, result_type='expand')
opp_last10 = opp_last10.rename(columns={0: "opp_wins_in_last10", 1: "opp_losses_in_last10", 2: "opp_ties_in_last10",
                                         3: "opp_goals_in_last10", 4: "opp_goals_against_in_last10"})
matches[["opp_wins_in_last10", "opp_losses_in_last10", "opp_ties_in_last10", "opp_goals_in_last10",
        "opp_goals_against_in_last10"]] = opp_last10.astype(int)

display(matches)


,index,home,away,year,time,attendance,league,home_score,away_score,home_starting_1,home_starting_2,home_starting_3,home_starting_4,home_starting_5,home_starting_6,home_starting_7,home_starting_8,home_starting_9,home_starting_10,home_starting_11,away_starting_1,away_starting_2,away_starting_3,away_starting_4,away_starting_5,away_starting_6,away_starting_7,away_starting_8,away_starting_9,away_starting_10,away_starting_11,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10
0,6462,Manchester Utd,Leicester City,2018,19:00,74439.0,2017-18,2,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2018-08-10 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,6463,Newcastle Utd,Tottenham,2018,11:30,51749.0,2017-18,1,2,Martin Dubravka,Ciaran Clark,Jamaal Lascelles,Paul Dummett,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Joselu,Ayoze Perez,Hugo Lloris,Jan Vertonghen,Davinson Sánchez,Ben Davies,Serge Aurier,Eric Dier,Dele Alli,Moussa Sissoko,Harry Kane,Christian Eriksen,Lucas Moura,1.88,3.53,4.37,1.98,1.83,11,8,2018-08-11 11:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6466,Huddersfield,Chelsea,2018,14:00,24121.0,2017-18,0,3,Ben Hamer,Christopher Schindler,Terence Kongolo,Mathias Jørgensen,Phillip Biling,Aaron Mooy,Jonathan Hogg,Chris Löwe,Florent Hadergjonaj,Steve Mounié,Alex Pritchard,Kepa Arrizabalaga,David Luiz,Antonio Rüdiger,Marcos Alonso,César Azpilicueta,Jorginho,Ross Barkley,N'Golo Kanté,Álvaro Morata,Willian,Pedro,3.83,3.40,2.05,2.01,1.81,11,8,2018-08-11 14:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6467,Watford,Brighton,2018,14:00,20051.0,2017-18,2,0,Ben Foster,Christian Kabasele,Craig Cathcart,José Holebas,Daryl Janmaat,Étienne Capoue,Abdoulaye Doucouré,Troy Deeney,Andre Gray,Roberto Pereyra,Will Hughes,Mat Ryan,Lewis Dunk,Shane Duffy,Bernardo,Bruno,Davy Propper,Dale Stephens,Solly March,Anthony Knockaert,Glenn Murray,Pascal Gross,2.36,3.14,3.31,2.35,1.59,11,8,2018-08-11 14:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,6469,Liverpool,West Ham,2018,12:30,53235.0,2017-18,4,0,Alisson,Virgil van Dijk,Joe Gomez,Andy Robertson,Trent Alexander-Arnold,Georginio Wijnaldum,Naby Keita,James Milner,Roberto Firmino,Sadio Mané,Mohamed Salah,Lukasz Fabianski,Angelo Ogbonna,Fabián Balbuena,Arthur Masuaku,Ryan Fredericks,Jack Wilshere,Declan Rice,Mark Noble,Marko Arnautovic,Felipe Anderson,Michail Antonio,3.83,3.80,1.92,1.55,2.42,12,8,2018-08-12 12:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,7714,Burnley,Crystal Palace,2021,15:00,18028.0,2020-21,3,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,20,11,2021-11-20 15:00:00,1,1,3,7,6,1,1,3,7,7,3,2,4,14,10,3,2,5,16,13
1257,7719,Liverpool,Arsenal,2021,17:30,53092.0,2020-21,4,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberla

In [8]:
# cleaning matches data

from datetime import datetime

df = pd.DataFrame(matches)
df = df.rename(columns={
                    "home":"team", 
                    "away":"opponent", 
                    "time (utc)":"time", 
                    "home_score":"score", 
                    "away_score": "opp_score",
                    "AvgH":"win_odds",
                    "AvgD":"draw_odds",
                    "AvgA":"loss_odds",
                    "Avg>2.5":"over_2.5goals_odds",
                    "Avg<2.5":"under_2.5goals_odds"})
new = {}
for i in range(1, 12):
    new["home_starting_" + str(i)] = "starting_" + str(i)
for i in range(1, 12):
    new["away_starting_" + str(i)] = "opp_starting_" + str(i)
df = df.rename(columns=new)
df['is_home'] = 1
# attendance has some weird NaNs that I couldn't get rid of so this doesn't work
df['attendance'] = df['attendance'].astype(int)

df.tail()

,index,team,opponent,year,time,attendance,league,score,opp_score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home
1252,7714,Burnley,Crystal Palace,2021,15:00,18028,2020-21,3,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,20,11,2021-11-20 15:00:00,1,1,3,7,6,1,1,3,7,7,3,2,4,14,10,3,2,5,16,13,1
1257,7719,Liverpool,Arsenal,2021,17:30,53092,2020-21,4,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberlain,Diogo Jota,Sadio Mané,Mohamed Salah,Aaron Ramsdale,Gabriel,Ben White,Nuno Tavares,Takehiro Tomiyasu,Albert Sambi Lokonga,Thomas Partey,Emile Smith Rowe,Bukayo Saka,Pierre-Emerick Aubameyang,Alexandre Lacazette,1.48,4.77,6.38,1.49,2.64,20,11,2021-11-20 17:30:00,2,1,2,10,11,2,1,2,8,6,3,2,3,13,15,3,3,2,13,9,1
1258,7720,Manchester City,Everton,2021,14:00,52571,2020-21,3,0,Ederson,Aymeric Laporte,John Stones,João Cancelo,Kyle Walker,Rodri,Ilkay Gündogan,Bernardo Silva,Cole Palmer,Phil Foden,Raheem Sterling,Jordan Pickford,Michael Keane,Ben Godfrey,Lucas Digne,Seamus Coleman,Fabian Delph,Allan,Demarai Gray,Anthony Gordon,Richarlison,Andros Townsend,1.18,7.56,15.95,1.49,2.61,21,11,2021-11-21 14:00:00,1,3,1,5,10,1,2,2,3,6,2,6,2,10,13,4,2,3,14,9,1
1259,7721,Tottenham,Leeds United,2021,16:30,58989,2020-21,2,1,Hugo Lloris,Eric Dier,Ben Davies,Japhet Tanganga,Harry Winks,Pierre-Emile Hojbjerg,Sergio Reguilón,Emerson,Harry Kane,Son Heung-Min,Lucas Moura,Illan Meslier,Kalvin Phillips,Liam Cooper,Diego Llorente,Pascal Struijk,Stuart Dallas,Mateusz Klich,Adam Forshaw,Jack Harrison,Daniel James,Joe Gelhardt,1.73,4.07,4.56,1.76,2.09,21,11,2021-11-21 16:30:00,2,2,1,5,7,1,1,3,5,5,5,4,1,12,12,1,3,5,9,16,1
914,7376,Newcastle Utd,Liverpool,2021,20:00,0,2019-20,0,0,Karl Darlow,Fabian Schär,Ciaran Clark,Federico Fernández,Matt Ritchie,DeAndre Yedlin,Matthew Longstaff,Isaac Hayden,Joelinton,Jacob Murphy,Callum Wilson,Alisson,Fabinho,Nathaniel Phillips,Andy Robertson,Trent Alexander-Arnold,Jordan Henderson,James Milner,Curtis Jones,Roberto Firmino,Sadio Mané,Mohamed Salah,9.81,5.93,1.29,1.50,2.60,30,12,2021-12-30 20:00:00,1,3,1,7,11,2,2,1,7,8,3,4,3,17,18,4,5,1,11,14,1


In [9]:
# prepare reverse matches and drop opp_score from matches

df2 = pd.DataFrame(df)
df = df.drop('opp_score', axis=1)
df2 = df2.drop('score', axis=1)
df2 = df2.rename(columns={"opp_score":"score"})

In [10]:
# finalize reverse matches

df2 = df2.rename(columns={"team":"opponent", 
                        "opponent":"team",
                         "win_odds":"loss_odds",
                         "loss_odds":"win_odds",
                         "wins_in_last5":"opp_wins_in_last5",
                         "losses_in_last5":"opp_losses_in_last5",
                         "ties_in_last5":"opp_ties_in_last5",
                         "goals_in_last5":"opp_goals_in_last5",
                         "goals_against_in_last5":"opp_goals_against_in_last5",
                        "opp_wins_in_last5":"wins_in_last5",
                         "opp_losses_in_last5":"losses_in_last5",
                         "opp_ties_in_last5":"ties_in_last5",
                         "opp_goals_in_last5":"goals_in_last5",
                         "opp_goals_against_in_last5":"goals_against_in_last5",
                        "wins_in_last10":"opp_wins_in_last10",
                         "losses_in_last10":"opp_losses_in_last10",
                         "ties_in_last10":"opp_ties_in_last10",
                         "goals_in_last10":"opp_goals_in_last10",
                         "goals_against_in_last10":"opp_goals_against_in_last10",
                        "opp_wins_in_last10":"wins_in_last10",
                         "opp_losses_in_last10":"losses_in_last10",
                         "opp_ties_in_last10":"ties_in_last10",
                         "opp_goals_in_last10":"goals_in_last10",
                         "opp_goals_against_in_last10":"goals_against_in_last10",
                         })
new = {}
df2['is_home'] = 0
for i in range(1, 12):
    new["opp_starting_" + str(i)] = "starting_" + str(i)
for i in range(1, 12):
    new["starting_" + str(i)] = "opp_starting_" + str(i)
df2 = df2.rename(columns=new)

df2.tail()

,index,opponent,team,year,time,attendance,league,score,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,loss_odds,draw_odds,win_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,is_home
1252,7714,Burnley,Crystal Palace,2021,15:00,18028,2020-21,3,Nick Pope,Ben Mee,James Tarkowski,Charlie Taylor,Matthew Lowton,Josh Brownhill,Ashley Westwood,Dwight McNeil,Johann Gudmundsson,Maxwel Cornet,Chris Wood,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,Cheikhou Kouyaté,Conor Gallagher,Christian Benteke,Wilfried Zaha,Jordan Ayew,2.77,3.23,2.70,2.18,1.70,20,11,2021-11-20 15:00:00,1,1,3,7,6,1,1,3,7,7,3,2,4,14,10,3,2,5,16,13,0
1257,7719,Liverpool,Arsenal,2021,17:30,53092,2020-21,0,Alisson,Virgil van Dijk,Joël Matip,Konstantinos Tsimikas,Trent Alexander-Arnold,Fabinho,Thiago Alcántara,Alex Oxlade-Chamberlain,Diogo Jota,Sadio Mané,Mohamed Salah,Aaron Ramsdale,Gabriel,Ben White,Nuno Tavares,Takehiro Tomiyasu,Albert Sambi Lokonga,Thomas Partey,Emile Smith Rowe,Bukayo Saka,Pierre-Emerick Aubameyang,Alexandre Lacazette,1.48,4.77,6.38,1.49,2.64,20,11,2021-11-20 17:30:00,2,1,2,10,11,2,1,2,8,6,3,2,3,13,15,3,3,2,13,9,0
1258,7720,Manchester City,Everton,2021,14:00,52571,2020-21,0,Ederson,Aymeric Laporte,John Stones,João Cancelo,Kyle Walker,Rodri,Ilkay Gündogan,Bernardo Silva,Cole Palmer,Phil Foden,Raheem Sterling,Jordan Pickford,Michael Keane,Ben Godfrey,Lucas Digne,Seamus Coleman,Fabian Delph,Allan,Demarai Gray,Anthony Gordon,Richarlison,Andros Townsend,1.18,7.56,15.95,1.49,2.61,21,11,2021-11-21 14:00:00,1,3,1,5,10,1,2,2,3,6,2,6,2,10,13,4,2,3,14,9,0
1259,7721,Tottenham,Leeds United,2021,16:30,58989,2020-21,1,Hugo Lloris,Eric Dier,Ben Davies,Japhet Tanganga,Harry Winks,Pierre-Emile Hojbjerg,Sergio Reguilón,Emerson,Harry Kane,Son Heung-Min,Lucas Moura,Illan Meslier,Kalvin Phillips,Liam Cooper,Diego Llorente,Pascal Struijk,Stuart Dallas,Mateusz Klich,Adam Forshaw,Jack Harrison,Daniel James,Joe Gelhardt,1.73,4.07,4.56,1.76,2.09,21,11,2021-11-21 16:30:00,2,2,1,5,7,1,1,3,5,5,5,4,1,12,12,1,3,5,9,16,0
914,7376,Newcastle Utd,Liverpool,2021,20:00,0,2019-20,0,Karl Darlow,Fabian Schär,Ciaran Clark,Federico Fernández,Matt Ritchie,DeAndre Yedlin,Matthew Longstaff,Isaac Hayden,Joelinton,Jacob Murphy,Callum Wilson,Alisson,Fabinho,Nathaniel Phillips,Andy Robertson,Trent Alexander-Arnold,Jordan Henderson,James Milner,Curtis Jones,Roberto Firmino,Sadio Mané,Mohamed Salah,9.81,5.93,1.29,1.50,2.60,30,12,2021-12-30 20:00:00,1,3,1,7,11,2,2,1,7,8,3,4,3,17,18,4,5,1,11,14,0


In [11]:
# combine matches and reversed matches

combined_df = df.append(df2)
combined_df.index = range(len(combined_df.index))
combined_df[(combined_df.attendance == "74439")]

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home


In [12]:
# merge matches and teams' stats

pre_final = pd.merge(combined_df, full_teamtable, on=['team', 'league'])
pre_final.head()

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Best Player,Top Scorer,Top Scorer Goals
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2018-08-10 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
1,6491,Manchester Utd,Tottenham,2018,19:00,744,2017-18,0,David de Gea,Chris Smalling,Phil Jones,Ander Herrera,Fred,Paul Pogba,Nemanja Matic,Luke Shaw,Antonio Valencia,Romelu Lukaku,Jesse Lingard,Hugo Lloris,Jan Vertonghen,Toby Alderweireld,Mousa Dembélé,Danny Rose,Kieran Trippier,Christian Eriksen,Dele Alli,Eric Dier,Harry Kane,Lucas Moura,2.56,3.27,2.86,2.03,1.79,27,8,2018-08-27 19:00:00,2,0,0,5,3,1,0,0,2,1,2,0,0,5,3,1,0,0,2,1,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
2,6538,Manchester Utd,Newcastle Utd,2018,16:30,74519,2017-18,3,David de Gea,Chris Smalling,Eric Bailly,Luke Shaw,Ashley Young,Nemanja Matic,Paul Pogba,Scott McTominay,Romelu Lukaku,Anthony Martial,Marcus Rashford,Martin Dubravka,Federico Fernández,Jamaal Lascelles,Javier Manquillo,DeAndre Yedlin,Mohamed Diamé,Jonjo Shelvey,Kenedy,Matt Ritchie,Yoshinori Muto,Ayoze Perez,2.13,3.55,3.45,1.77,2.06,6,10,2018-10-06 16:30:00,1,4,0,6,11,1,3,1,4,7,3,3,0,9,11,1,4,1,5,9,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.6,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.1,0.79,1.88,1.08,1.86,513,176,34.3,13.5,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16
3,6560,Manchester Utd,Everton,2018,16:00,74525,2017-18,2,David de Gea,Victor Lindelöf,Chris Smalling,Luke Shaw,Ashley Young,Nemanja Matic,Paul Pogba,Fred,Marcus Rashford,Anthony Martial,Juan Mata,Jordan Pickford,Kurt Zouma,Michael Keane,Lucas Digne,Seamus Coleman,Gylfi Sigurdsson,Idrissa Gueye,André Gomes,Richarlison,Bernard,Theo Walcott,1.73,3.80,4.93,1.81,2.01,28,10,2018-10-28 16:00:00,3,1,1,11,7,2,2,1,7,6,4,3,1,14,15,3,2,2,11,9,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.5,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,

In [13]:
# create teamtable with alternative column names

opp_full_teamtable = pd.DataFrame(full_teamtable)
new = {}
for col in opp_full_teamtable.columns:
    new[str(col)] = "opp_" + str(col)
opp_full_teamtable = opp_full_teamtable.rename(columns=new)

opp_full_teamtable = opp_full_teamtable.rename(columns={"opp_team":"opponent", "opp_league":"league"})
opp_full_teamtable.head()

,opp_Rk,opponent,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_Goalkeeper,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,league,opp_Best Player,opp_Top Scorer,opp_Top Scorer Goals
0,6,Arsenal,19,6,13,74,51,23,63,68.4,50.6,17.8,0.47,Petr Čech,30,26.8,61.5,73,61,69,4,5,1.92,1.61,3.53,1.82,3.42,64.6,1.80,1.35,3.15,1.70,3.05,51,38,46,5,6,52,4,1.34,1.00,2.34,1.21,2.21,50.6,46.1,34.0,80.1,1.33,0.89,2.23,1.21,2.11,590,227,38.5,15.53,5.97,0.12,0.30,17.4,30,0.11,4.6,4.4,2017-18,Aaron Ramsey,Alexandre Lacazette,14
1,12,Bournemouth,11,11,16,45,61,-16,44,44.8,61.4,-16.6,-0.44,Asmir Begović,22,26.9,47.8,45,36,42,3,3,1.18,0.95,2.13,1.11,2.05,42.6,1.18,0.85,2.03,1.12,1.97,60,44,58,2,3,73,1,1.58,1.16,2.74,1.53,2.68,61.4,59.3,43.7,103.0,1.62,1.15,2.77,1.56,2.71,473,153,32.3,12.45,4.03,0.09,0.27,17.5,13,0.09,0.2,-0.6,2017-18,Nathan Ake,Callum Wilson,8
2,15,Brighton,9,13,16,34,54,-20,40,37.2,52.6,-15.5,-0.41,Mathew Ryan,24,27.5,43.7,33,25,28,5,7,0.87,0.66,1.53,0.74,1.39,31.8,0.98,0.60,1.58,0.84,1.44,49,36,48,1,3,30,2,1.29,0.95,2.24,1.26,2.21,52.6,50.3,34.5,84.9,1.38,0.91,2.29,1.32,2.23,381,113,29.7,10.03,2.97,0.07,0.25,17.9,3,0.09,-4.2,-3.8,2017-18,Pascal Groß,Glenn Murray,12
3,7,Burnley,14,12,12,36,39,-3,54,34.5,51.7,-17.3,-0.45,Nick Pope,24,27.5,43.9,35,26,35,0,0,0.92,0.68,1.61,0.92,1.61,34.5,0.91,0.63,1.53,0.91,1.53,38,29,35,3,5,45,6,1.00,0.76,1.76,0.92,1.68,51.7,48.2,37.4,85.7,1.36,0.99,2.35,1.27,2.25,387,126,32.6,10.18,3.32,0.09,0.28,16.0,16,0.09,0.5,0.5,2017-18,Nick Pope,Chris Wood,10
4,5,Chelsea,21,7,10,62,38,24,70,57.9,35.3,22.7,0.60,Thibaut Courtois,26,26.7,55.4,60,42,57,3,3,1.58,1.11,2.68,1.50,2.61,55.7,1.52,1.11,2.64,1.46,2.58,36,26,34,2,2,75,1,0.95,0.68,1.63,0.89,1.58,35.3,33.8,25.4,59.1,0.93,0.67,1.60,0.89,1.56,602,209,34.7,15.84,5.50,0.09,0.27,17.4,19,0.09,2.1,1.3,2017-18,N'Golo Kanté,Eden Hazard,12


In [14]:
# merge matches and oppenents' stats

final = pd.merge(pre_final, opp_full_teamtable, on=['opponent', 'league'])
final.head()

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Goalkeeper,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Best Player,Top Scorer,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_Goalkeeper,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Best Player,opp_Top Scorer,opp_Top Scorer Goals
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2018-08-10 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,Kasper Schmeichel,27,27.2,47.9,53,38,48,5,6,1.39,1.0,2.39,1.26,2.26,43.6,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.0,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,Harry Maguire,Jamie Vardy,20
1,6709,Manchester Utd,Leicester City,2019,14:05,32148,2017-18,1,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Ashley Young,Nemanja Matic,Ander Herrera,Alexis Sánchez,Marcus Rashford,Paul Pogba,Jesse Lingard,Kasper Schmeichel,Harry Maguire,Jonny Evans,Ben Chilwell,Ricardo Pereira,James Maddison,Wilfred Ndidi,Nampalys Mendy,Jamie Vardy,Harvey Barnes,Demarai Gray,2.11,3.41,3.63,1.81,2.02,3,2,2019-02-03 14:05:00,2,2,1,8,7,2,2,1,5,5,4,2,4,18,13,3,4,3,8,9,0,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,David de Gea,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,David de Gea,Romelu Lukaku,16,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,Kasper Schmeichel,27,27.2,47.9,53,38,48,5,6,1.39,1.0,2.39,1.26,2.26,43.6,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.0,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,Harry Maguire,Jamie Vardy,20
2,65

In [15]:
# is team top scorer playing

final.loc[((final.starting_1 == final["Top Scorer"]) | (final.starting_2 == final["Top Scorer"]) |
           (final.starting_3 == final["Top Scorer"]) | (final.starting_4 == final["Top Scorer"]) |
           (final.starting_5 == final["Top Scorer"]) | (final.starting_6 == final["Top Scorer"]) |
           (final.starting_7 == final["Top Scorer"]) | (final.starting_8 == final["Top Scorer"]) |
           (final.starting_9 == final["Top Scorer"]) | (final.starting_10 == final["Top Scorer"]) |
           (final.starting_11 == final["Top Scorer"])), "Top Scorer playing"] = 1
final.loc[(final["Top Scorer playing"] != 1), "Top Scorer playing"] = 0

#is opponent top scorer playing
final.loc[((final.opp_starting_1 == final["opp_Top Scorer"]) | (final.opp_starting_2 == final["opp_Top Scorer"]) |
           (final.opp_starting_3 == final["opp_Top Scorer"]) | (final.opp_starting_4 == final["opp_Top Scorer"]) |
           (final.opp_starting_5 == final["opp_Top Scorer"]) | (final.opp_starting_6 == final["opp_Top Scorer"]) |
           (final.opp_starting_7 == final["opp_Top Scorer"]) | (final.opp_starting_8 == final["opp_Top Scorer"]) |
           (final.opp_starting_9 == final["opp_Top Scorer"]) | (final.opp_starting_10 == final["opp_Top Scorer"]) |
           (final.opp_starting_11 == final["opp_Top Scorer"])), "opp_Top Scorer playing"] = 1
final.loc[(final["opp_Top Scorer playing"] != 1), "opp_Top Scorer playing"] = 0

# is team best player playing
final.loc[((final.starting_1 == final["Best Player"]) | (final.starting_2 == final["Best Player"]) |
           (final.starting_3 == final["Best Player"]) | (final.starting_4 == final["Best Player"]) |
           (final.starting_5 == final["Best Player"]) | (final.starting_6 == final["Best Player"]) |
           (final.starting_7 == final["Best Player"]) | (final.starting_8 == final["Best Player"]) |
           (final.starting_9 == final["Best Player"]) | (final.starting_10 == final["Best Player"]) |
           (final.starting_11 == final["Best Player"])), "Best Player playing"] = 1
final.loc[(final["Best Player playing"] != 1), "Best Player playing"] = 0

#is opponent best player playing
final.loc[((final.opp_starting_1 == final["opp_Best Player"]) | (final.opp_starting_2 == final["opp_Best Player"]) |
           (final.opp_starting_3 == final["opp_Best Player"]) | (final.opp_starting_4 == final["opp_Best Player"]) |
           (final.opp_starting_5 == final["opp_Best Player"]) | (final.opp_starting_6 == final["opp_Best Player"]) |
           (final.opp_starting_7 == final["opp_Best Player"]) | (final.opp_starting_8 == final["opp_Best Player"]) |
           (final.opp_starting_9 == final["opp_Best Player"]) | (final.opp_starting_10 == final["opp_Best Player"]) |
           (final.opp_starting_11 == final["opp_Best Player"])), "opp_Best Player playing"] = 1
final.loc[(final["opp_Best Player playing"] != 1), "opp_Best Player playing"] = 0

# is team goalkeeper playing
final.loc[((final.starting_1 == final["Goalkeeper"]) | (final.starting_2 == final["Goalkeeper"]) |
           (final.starting_3 == final["Goalkeeper"]) | (final.starting_4 == final["Goalkeeper"]) |
           (final.starting_5 == final["Goalkeeper"]) | (final.starting_6 == final["Goalkeeper"]) |
           (final.starting_7 == final["Goalkeeper"]) | (final.starting_8 == final["Goalkeeper"]) |
           (final.starting_9 == final["Goalkeeper"]) | (final.starting_10 == final["Goalkeeper"]) |
           (final.starting_11 == final["Goalkeeper"])), "Goalkeeper playing"] = 1
final.loc[(final["Goalkeeper playing"] != 1), "Goalkeeper playing"] = 0

#is opponent goalkeeper playing
final.loc[((final.opp_starting_1 == final["opp_Goalkeeper"]) | (final.opp_starting_2 == final["opp_Goalkeeper"]) |
           (final.opp_starting_3 == final["opp_Goalkeeper"]) | (final.opp_starting_4 == final["opp_Goalkeeper"]) |
           (final.opp_starting_5 == final["opp_Goalkeeper"]) | (final.opp_starting_6 == final["opp_Goalkeeper"]) |
           (final.opp_starting_7 == final["opp_Goalkeeper"]) | (final.opp_starting_8 == final["opp_Goalkeeper"]) |
           (final.opp_starting_9 == final["opp_Goalkeeper"]) | (final.opp_starting_10 == final["opp_Goalkeeper"]) |
           (final.opp_starting_11 == final["opp_Goalkeeper"])), "opp_Goalkeeper playing"] = 1
final.loc[(final["opp_Goalkeeper playing"] != 1), "opp_Goalkeeper playing"] = 0


#converting to int
final["Top Scorer playing"] = final["Top Scorer playing"].astype(int)
final["opp_Top Scorer playing"] = final["opp_Top Scorer playing"].astype(int)
final["Best Player playing"] = final["Best Player playing"].astype(int)
final["opp_Best Player playing"] = final["opp_Best Player playing"].astype(int)
final["Goalkeeper playing"] = final["Goalkeeper playing"].astype(int)
final["opp_Goalkeeper playing"] = final["opp_Goalkeeper playing"].astype(int)

final = final.drop(["Top Scorer", "Best Player", "Goalkeeper", "opp_Best Player", "opp_Goalkeeper", "opp_Top Scorer"], axis=1)
final.sort_values("index", ascending=True,inplace=True)

display(final.head())


,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing
0,6462,Manchester Utd,Leicester City,2018,19:00,74439,2017-18,2,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,1.56,3.92,7.06,2.03,1.79,10,8,2018-08-10 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,16,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,27,27.2,47.9,53,38,48,5,6,1.39,1.00,2.39,1.26,2.26,43.6,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.00,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,20,0,0,1,1,1,1
541,6462,Leicester City,Manchester Utd,2018,19:00,74439,2017-18,1,Kasper Schmeichel,Harry Maguire,Wes Morgan,Ben Chilwell,Daniel Amartey,James Maddison,Wilfred Ndidi,Adrien Silva,Kelechi Iheanacho,Demarai Gray,Ricardo Pereira,David de Gea,Victor Lindelöf,Eric Bailly,Luke Shaw,Matteo Darmian,Andreas Pereira,Paul Pogba,Fred,Marcus Rashford,Alexis Sánchez,Juan Mata,7.06,3.92,1.56,2.03,1.79,10,8,2018-08-10 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,12,11,15,56,60,-4,47,48.1,49.9,-1.8,-0.05,27,27.2,47.9,53,38,48,5,6,1.39,1.00,2.39,1.26,2.26,43.6,1.27,0.84,2.11,1.15,1.99,58,47,56,2,4,52,1,1.53,1.24,2.76,1.47,2.71,49.9,46.9,37.1,83.9,1.31,0.98,2.29,1.23,2.21,418,137,32.8,11.00,3.61,0.11,0.35,17.0,12,0.11,4.9,4.4,20,2,25,6,7,68,28,40,81,60.7,41.7,19.0,0.50,27,26.7,54.8,67,53,66,1,3,1.76,1.39,3.16,1.74,3.13,58.2,1.60,1.15,2.75,1.53,2.69,26,18,25,1,1,69,1,0.68,0.47,1.16,0.66,1.13,41.7,41.0,29.8,70.8,1.10,0.79,1.88,1.08,1.86,513,176,34.3,13.50,4.63,0.13,0.38,17.6,22,0.12,6.3,7.8,16,0,0,1,1,1,1
34,6463,Newcastle Utd,Tottenham,2018,11:30,51749,2017-18,1,Martin Dubravka,Ciaran Clark,Jamaal Lasce

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [17]:
import random
# Split data into train, validation and test with approximate 70-10-20 split
# Using a non-traditonal way of splitting to get both sides from matches for the test set, so odds and results can be calculated
def getModelData():
    current = final[final.league == '2020-21']
    indexes = current['index'].tolist()
    list_set = set(indexes)
    indexes = (list(list_set))
    X_test = current[current['index'] == indexes[0]]
    y_test = current[current['index'] == indexes[0]]['score']
    X_val = current[current['index'] == indexes[1]]
    y_val = current[current['index'] == indexes[1]]['score']
    # Current season will have a bigger percent going into test, so our results can be indicative of model predicting current football
    for i in range(2,len(indexes)):
        choose = random.randint(1,2)
        if choose == 1:
            X_test = X_test.append(current[current['index'] == indexes[i]])
            y_test = y_test.append(current[current['index'] == indexes[i]]['score'])
        else:
            X_val = X_val.append(current[current['index'] == indexes[i]])
            y_val = y_val.append(current[current['index'] == indexes[i]]['score'])
    X = final[final.league != '2020-21']
    indexes2 = X['index'].tolist()
    list_set = set(indexes2)
    indexes2 = (list(list_set))
    for i in range(len(indexes2)):
        choose = random.randint(1,83)
        if choose < 11:
            X_test = X_test.append(X[X['index'] == indexes2[i]])
            y_test = y_test.append(X[X['index'] == indexes2[i]]['score'])
        else:
            X_val = X_val.append(X[X['index'] == indexes2[i]])
            y_val = y_val.append(X[X['index'] == indexes2[i]]['score'])
    X_train,X_val,y_train,y_val = train_test_split(X_val, y_val, test_size=0.18, random_state=42)
    #Will copy test dataset with all columns to use it when getting results
    X_test2 = X_test.copy(deep=True)
    # All dropped columns either have a non-numeric value or are not stats
    X_test = X_test.drop(['attendance','index','score','team','year','day','month','opponent','time','league','starting_1','starting_2','starting_3','starting_4','starting_5','starting_6','starting_7','starting_8','starting_9','starting_10','starting_11','opp_starting_1','opp_starting_2','opp_starting_3','opp_starting_4','opp_starting_5','opp_starting_6','opp_starting_7','opp_starting_8','opp_starting_9','opp_starting_10','opp_starting_11','datetime'],axis=1)
    X_train = X_train.drop(['attendance','index','score','team','year','day','month','opponent','time','league','starting_1','starting_2','starting_3','starting_4','starting_5','starting_6','starting_7','starting_8','starting_9','starting_10','starting_11','opp_starting_1','opp_starting_2','opp_starting_3','opp_starting_4','opp_starting_5','opp_starting_6','opp_starting_7','opp_starting_8','opp_starting_9','opp_starting_10','opp_starting_11','datetime'],axis=1)
    X_val = X_val.drop(['attendance','index','score','team','year','day','month','opponent','time','league','starting_1','starting_2','starting_3','starting_4','starting_5','starting_6','starting_7','starting_8','starting_9','starting_10','starting_11','opp_starting_1','opp_starting_2','opp_starting_3','opp_starting_4','opp_starting_5','opp_starting_6','opp_starting_7','opp_starting_8','opp_starting_9','opp_starting_10','opp_starting_11','datetime'],axis=1)
    return X_train, y_train, X_val, y_val, X_test, y_test,X_test2
X_train, y_train, X_val, y_val, X_test, y_test,X_test2 = getModelData()

In [18]:
# Testing simple linear regression with normalized data
# With every model we will fit on training data, validate on validation data and will put the best models to the test
# on the test set
reg = LinearRegression(normalize=True).fit(X_train, y_train)
print("Simple linear regression MSE with all train data:\t\t\t", mean_squared_error(y_val,reg.predict(X_val)))

Simple linear regression MSE with all train data:			 1.4707458506670077


In [19]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [20]:
# Method to find on average best alpha for Ridge or Lasso regression
# NB! these methods provide an AVERAGE best alpha, which means with our random seed for choosing train and validaton data
# it may not necessarily be the best

from sklearn.model_selection import KFold

# Method taken from IDS2021_HW07 notebook
def get_best_alpha(model,X,y,alphas):
    kf = KFold(n_splits=3, shuffle=False) # had to remove random_state to avoid error
    results = []
    for alpha in alphas:
        model.set_params(alpha=alpha)
        for train_indexes, test_indexes in kf.split(X):
            X_test, X_val = X.values[train_indexes], X.values[test_indexes]
            y_test, y_val = y.values[train_indexes], y.values[test_indexes]

            model.fit(X_test, y_test)
            mse = mean_squared_error(y_val,model.predict(X_val))
            results.append((alpha, mse))
   

    best_alpha, mse = sorted(results, key=lambda x: x[1])[0]
    return best_alpha, mse

In [21]:
# Find on average best alphas for Ridge and Lasso regression

import numpy as np

ridge_alphas = np.linspace(0.001, 0.01, 100) 
lasso_alphas = np.linspace(0.1, 1, 100) 

print("Ridge best alpha %.4f - Avg MSE %.4f " % get_best_alpha(Ridge(), X_train.append(X_val), y_train.append(y_val), ridge_alphas))
print("Lasso best alpha %.4f - Avg MSE %.4f " % get_best_alpha(Lasso(), X_train.append(X_val), y_train.append(y_val), lasso_alphas))

Ridge best alpha 0.0058 - Avg MSE 1.4675 
Lasso best alpha 0.7909 - Avg MSE 1.3615 


In [27]:
# Testing Ridge regression

ridge_alpha = 0.0058
ridge = Ridge(alpha=ridge_alpha).fit(X_train, y_train)
print("Ridge regression MSE with alpha=" + str(ridge_alpha) + ":\t", mean_squared_error(y_val, ridge.predict(X_val)))

Ridge regression MSE with alpha=0.0058:	 1.4576420960521053


In [28]:
# Testing Lasso regression

lasso_alpha = 0.7909
lasso = Lasso(alpha=lasso_alpha).fit(X_train, y_train)
#print(lasso.coef_)
print("Lasso regression MSE with alpha=" + str(lasso_alpha) + ":\t", mean_squared_error(y_val, lasso.predict(X_val)))

Lasso regression MSE with alpha=0.7909:	 1.3663780082476011


In [29]:
# Find out non-zero coefficients in a regression model
def getParamsWithWeights(coef):
    weights = []
    for i in range(len(coef)):
        if (coef[i] != 0):
            weights.append([coef[i],X_train.columns[i]])
    return sorted(weights)
getParamsWithWeights(lasso.coef_)

[[-0.0029624003595367632, 'opp_SoT'],
 [-0.0007747514042056661, 'opp_Sh'],
 [0.002318712088411244, 'opp_npxG+xA'],
 [0.004211352352624075, 'GD'],
 [0.0059410367990244215, 'SoT']]

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [31]:
#RandomForestRegressor, found optimized parameters by trying out different values
rfr = RandomForestRegressor(n_estimators=10000, max_depth=1000,min_samples_leaf=20, n_jobs=4, random_state=24)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_val)
print("RandomForestRegressor, MSE:", mean_squared_error(y_val, y_pred))

RandomForestRegressor, MSE: 1.3874985501341242


In [32]:
#SVM with rbf kernel
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_sc = sc_X.fit_transform(X_train)
X_val_sc = sc_X.fit_transform(X_val)

svm = SVR(kernel = 'rbf', gamma='auto',)
svm.fit(X_train_sc, y_train)

y_pred = svm.predict(X_val_sc)

print("SVM rbf, MSE:", mean_squared_error(y_val, y_pred))

SVM rbf, MSE: 1.5252362321694102


In [33]:
#Method to find the best alpha and L1_ratio combo on the validation set
from sklearn.linear_model import ElasticNet
def get_best_alphaElasticNet(model,X,y,alphas):
    kf = KFold(n_splits=3, shuffle=False) # had to remove random_state to avoid error
    results = []
    for alpha in alphas:
        for l1_ratio in range(0,11):
            model.set_params(alpha=alpha,l1_ratio=l1_ratio*0.1,random_state=1,max_iter=50000)
            for train_indexes, test_indexes in kf.split(X):
                X_test, X_val = X.values[train_indexes], X.values[test_indexes]
                y_test, y_val = y.values[train_indexes], y.values[test_indexes]
                model.fit(X_test, y_test)
                mse = mean_squared_error(y_val,model.predict(X_val))
                results.append((alpha, l1_ratio, mse))
   

    best_alpha,l1_ratio,mse = sorted(results, key=lambda x: x[2])[0]
    return best_alpha,l1_ratio,mse

In [34]:
en_alphas = np.linspace(0.1, 1, 100)

#Here we used the function above, it took a long time to run it and it still caused a lot of convergence warnings
#print("ElasticNet best alpha %.4f, best L1 ratio %.4f - Avg MSE %.4f " % get_best_alphaElasticNet(ElasticNet(), X_train.append(X_val), y_train.append(y_val), en_alphas))

In [35]:
en_alpha = 0.2545
l1_ratio = 0.7
en = ElasticNet(alpha=en_alpha,l1_ratio=l1_ratio,random_state=1,max_iter=10000).fit(X_train, y_train)
print("ElasticNet regression MSE with alpha=" + str(en_alpha) + ":\t", mean_squared_error(y_val, en.predict(X_val)))

ElasticNet regression MSE with alpha=0.2545:	 1.3527158840630549


In [36]:
#Following Tensorflow code is based on https://www.tensorflow.org/tutorials/keras/regression

In [37]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [38]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

In [39]:
#Here we tried changing the layers and activation function
# Current code is the best solution we found
def build_and_compile_model(norm,activation):
  model = keras.Sequential([
      norm,
      layers.Dense(288, activation=activation),
      layers.Dense(144, activation=activation),
      layers.Dense(72, activation=activation),
      layers.Dense(1),
  ])

  model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [40]:
dnn_model = build_and_compile_model(normalizer,'sigmoid')
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=0, epochs=1000)
dnn_model.evaluate(X_val, y_val, verbose=0)

1.8487071990966797

In [41]:
#Finding each tried model's best variation's MSE on the test set
print(mean_squared_error(y_test, en.predict(X_test)))
print(mean_squared_error(y_test, lasso.predict(X_test)))
print(mean_squared_error(y_test, rfr.predict(X_test)))
print(dnn_model.evaluate(X_test, y_test, verbose=0))
print(mean_squared_error(y_test, ridge.predict(X_test)))
print(mean_squared_error(y_test, reg.predict(X_test)))
X_test_sc = sc_X.fit_transform(X_test)
print(mean_squared_error(y_test, svm.predict(X_test_sc)))

1.311241424474402
1.292807166615396
1.3364473975565299
2.056701183319092
1.4708233331124154
1.547252976455537
1.470799097259653


In [42]:
# Function to use the test set with all columns and add calculated goals column into it 
def addPred(y_pred):
    X_calc = X_test2.copy(deep=True)
    X_calc['expected'] = y_pred
    return X_calc

# We will use ElasticNet(en) first
y_pred = en.predict(X_test)
X_calc = addPred(y_pred)
# function to add the calculated goals and actual goals to each row with a home team
def getAway(home,index):
    score = 0
    for i in range(len(X_calc.columns)):
        if X_calc.columns[i] == 'score':
            score = i
            break
    home_id = 0
    for i in range(len(X_calc.columns)):
        if X_calc.columns[i] == 'is_home':
            home_id = i
            break
    if home:
        for j in range(len(X_calc)):
            if X_calc.iloc[j][0] == index and X_calc.iloc[j][home_id] == 0:
                return X_calc.iloc[j][-1],X_calc.iloc[j][score]
    return -1,-1
# Add these columns with the getAway function
exp = X_calc.apply(lambda x: getAway(x['is_home'], x['index']), axis=1, result_type='expand')
exp = exp.rename(columns={0: "away_expected",1:"away_score"})
X_calc[["away_expected","away_score"]] = exp.astype(float)
X_calc = X_calc[X_calc['away_expected'] != -1]

In [43]:
X_calc

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing,expected,away_expected,away_score
1736,7680,Newcastle Utd,Tottenham,2021,15:30,52214,2020-21,2,Karl Darlow,Ciaran Clark,Jamaal Lascelles,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Allan Saint-Maximin,Callum Wilson,Hugo Lloris,Eric Dier,Cristian Romero,Sergio Reguilón,Emerson,Tanguy Ndombele,Oliver Skipp,Pierre-Emile Hojbjerg,Harry Kane,Son Heung-Min,Lucas Moura,3.51,3.58,2.09,1.79,2.07,17,10,2021-10-17 15:30:00,2,1,2,9,7,4,1,0,9,5,3,1,2,13,9,4,2,0,9,6,1,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,7,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,23,1,1,1,1,1,1,1.084706,1.818192,3.0
1651,7684,Crystal Palace,Newcastle Utd,2021,14:00,24609,2020-21,1,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,James McArthur,Conor Gallagher,Christian Benteke,Odsonne Édouard,Michael Olise,Karl Darlow,Jamaal Lascelles,Ciaran Clark,Emil Krafth,Matt Ritchie,Javier Manquillo,Sean Longstaff,Ryan Fraser,Isaac Hayden,Allan Saint-Maximin,Callum Wilson,1.97,3.61,3.88,1.95,1.89,23,10,2021-10-23 14:00:00,2,0,3,11,5,2,1,2,11,8,2,1,4,13,10,3,2,2,15,12,1,14,12,8,18,41,66,-25,44,32.4,57.5,-25.0,-0.66,24,29.1,40.1,39,29,36,3,4,1.03,0.76,1.79,0.95,1.71,29.5,0.85,0.55,1.40,0.78,1.33,66,47,63,3,4,44,4,1.74,1.24,2.97,1.66,2.89,57.5,54.4,39.8,94.2,1.51,1.05,2.56,1.43,2.48,346,129,37.3,9.11,3.39,0.10,0.28,16.8,18,0.09,6.6,6.5,11,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,0,1,1,1

In [44]:
from scipy.stats import poisson
#Function to add desired results to this table
def profit(exp,away_exp,win_odds,draw_odds,loss_odds,over_odds,under_odds,score,away_score):
    home = []
    away = []
    # First we will use Poisson distribution to get probabilities of either team scoring certain amount of goals
    home.append(poisson.pmf(k=0, mu=exp))
    home.append(poisson.pmf(k=1, mu=exp))
    home.append(poisson.pmf(k=2, mu=exp))
    home.append(poisson.pmf(k=3, mu=exp))
    home.append(poisson.pmf(k=4, mu=exp))
    home.append(poisson.pmf(k=5, mu=exp))
    away.append(poisson.pmf(k=0, mu=away_exp))
    away.append(poisson.pmf(k=1, mu=away_exp))
    away.append(poisson.pmf(k=2, mu=away_exp))
    away.append(poisson.pmf(k=3, mu=away_exp))
    away.append(poisson.pmf(k=4, mu=away_exp))
    away.append(poisson.pmf(k=5, mu=away_exp))
    # Then we will get probabilites for each outcome by adding together probabilities of scores, both in terms of scoring 
    # over 2.5 goals and winning
    win = 0
    draw = 0
    loss = 0
    over = 0
    under = 0
    for i in range(6):
        for j in range(6):
            if i > j:
                win += home[i] * away[j]
            if i < j:
                loss += home[i] * away[j]
            if i == j:
                draw += home[i] * away[j]
            if i + j > 2:
                over += home[i] * away[j]
            if i + j <= 2:
                under += home[i] * away[j]
    #Convert each to decimal odds
    win = 1/win
    draw = 1/draw
    loss = 1/loss
    over = 1/over
    under = 1/under
    # Find which outcome is "underrated" by the bookmaker the most, meaning for which outcome our odds are the lowest compared to
    # bookmakers
    biggest = -100
    choice = ''
    choice_odds = 0
    if win_odds - win > biggest:
        biggest = win_odds - win
        choice = 'win'
        choice_odds = win_odds
    if draw_odds - draw > biggest:
        biggest = draw_odds - draw
        choice = 'draw'
        choice_odds = draw_odds
    if loss_odds - loss > biggest:
        biggest = loss_odds - loss
        choice = 'loss'
        choice_odds = loss_odds
    choice2 = 'under'
    choice_odds2 = under_odds
    over_biggest = under_odds - under
    if over_odds - over > under_odds - under:
        choice2 = 'over'
        over_biggest = over_odds - over
        choice_odds2 = over_odds
    # Find out, what was the actual result and if we won with our method
    outcome1 = ''
    if score > away_score:
        outcome1 = 'win'
    elif score < away_score:
        outcome1 = 'loss'
    else:
        outcome1 = 'draw'
    outcome2 = 'under'
    if score + away_score > 2:
        outcome2 = 'over'
    # Find, how much we won or did we lose 10 euros (for each predcition we are putting down 10 euros)
    profit1 = -10
    profit2 = -10
    if outcome1 == choice:
        profit1 = 10 * choice_odds - 10
    if outcome2 == choice2:
        profit2 = 10 * choice_odds2 - 10
    # Now we will try predicting, what the model actuall predicted to be the outcome
    most_likely = win_odds
    most_likely_choice = 'win'
    if loss < win and loss < draw:
        most_likely = loss_odds
        most_likely_choice = 'loss'
    if draw < win and draw < loss:
        most_likely = draw_odds
        most_likely_choice = 'draw'
    most_likely2 = under_odds
    most_likely_choice2 = 'under'
    if over < under:
        most_likely2 = over_odds
        most_likely_choice2 = 'over'
    # Again find the monetary result
    profit3 = -10
    profit4 = -10
    if outcome1 == most_likely_choice:
        profit3 = 10 * most_likely - 10
    if outcome2 == most_likely_choice2:
        profit4 = 10 * most_likely2 - 10
    
    #Return odds our choices with the underrated method and our choices with the sensible method
    return win,draw,loss,over,under,choice,profit1,choice2,profit2,profit3,profit4

In [45]:
#Add all of these to the dataset
outcome = X_calc.apply(lambda x: profit(x['expected'], x['away_expected'],x['win_odds'],x['draw_odds'],x['loss_odds'],x['over_2.5goals_odds'],x['under_2.5goals_odds'],x['score'],x['away_score']), axis=1, result_type='expand')
outcome = outcome.rename(columns={0: "win_odds_pred",1:"draw_odds_pred",2:"loss_odds_pred",3:"over_odds_pred",4:"under_odds_pred",5:"pred",6:"pred_profit",7:"goals_pred",8:"goals_pred_profit",9:"sensible_profit",10:"sensible_profit_goals"})
X_calc[["win_odds_pred","draw_odds_pred","loss_odds_pred","over_odds_pred","under_odds_pred","pred","pred_profit","goals_pred","goals_pred_profit","sensible_profit","sensible_profit_goals"]] = outcome

In [46]:
X_calc

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing,expected,away_expected,away_score,win_odds_pred,draw_odds_pred,loss_odds_pred,over_odds_pred,under_odds_pred,pred,pred_profit,goals_pred,goals_pred_profit,sensible_profit,sensible_profit_goals
1736,7680,Newcastle Utd,Tottenham,2021,15:30,52214,2020-21,2,Karl Darlow,Ciaran Clark,Jamaal Lascelles,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Allan Saint-Maximin,Callum Wilson,Hugo Lloris,Eric Dier,Cristian Romero,Sergio Reguilón,Emerson,Tanguy Ndombele,Oliver Skipp,Pierre-Emile Hojbjerg,Harry Kane,Son Heung-Min,Lucas Moura,3.51,3.58,2.09,1.79,2.07,17,10,2021-10-17 15:30:00,2,1,2,9,7,4,1,0,9,5,3,1,2,13,9,4,2,0,9,6,1,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,7,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,23,1,1,1,1,1,1,1.084706,1.818192,3.0,4.463505,4.354226,1.870739,1.841769,2.245712,loss,10.9,over,7.9,10.9,7.9
1651,7684,Crystal Palace,Newcastle Utd,2021,14:00,24609,2020-21,1,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,James McArthur,Conor Gallagher,Christian Benteke,Odsonne Édouard,Michael Olise,Karl Darlow,Jamaal Lascelles,Ciaran Clark,Emil Krafth,Matt Ritchie,Javier Manquillo,Sean Longstaff,Ryan Fraser,Isaac Hayden,Allan Saint-Maximin,Callum Wilson,1.97,3.61,3.88,1.95,1.89,23,10,2021-10-23 14:00:00,2,0,3,11,5,2,1,2,11,8,2,1,4,13,10,3,2,2,15,12,1,14,12,8,18,41,66,-25,44,32.4,57.5,-25.0,-0.66,24,29.1,40.1,39,29,36,3,4,1.03,0.76,1.79,0.95,1.71,29.5,0.85,0.55,1.40,0.78,1.33,66,47,63,3,4,44,4,1.74,1.24,2.97,1.66,2.89,57.5,54.4,39.8,94.2,1.51,1.05,2.56,1.43,2.48,346,129,37.3,9.11,3.39,0.10,0.28,16.8,18,0.09,6.6,6.5,11,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27

In [49]:
#Calculate outcome of each method with both winner and goals prediction
print(X_calc['pred_profit'].sum())
print(X_calc['goals_pred_profit'].sum())
print(X_calc['sensible_profit'].sum())
print(X_calc['sensible_profit_goals'].sum())
#Also overall return on investment  
print((X_calc['sensible_profit_goals'].sum() + X_calc['sensible_profit'].sum() + X_calc['goals_pred_profit'].sum() + X_calc['pred_profit'].sum()) /(len(X_calc)*40))

522.1999999999999
125.20000000000003
87.90000000000003
-37.00000000000003
0.10776234567901234


In [50]:
#Use lasso next
en_results = X_calc.copy(deep=True)
y_pred = lasso.predict(X_test)
X_calc = addPred(y_pred)


exp = X_calc.apply(lambda x: getAway(x['is_home'], x['index']), axis=1, result_type='expand')
exp = exp.rename(columns={0: "away_expected",1:"away_score"})
X_calc[["away_expected","away_score"]] = exp.astype(float)
X_calc = X_calc[X_calc['away_expected'] != -1]
outcome = X_calc.apply(lambda x: profit(x['expected'], x['away_expected'],x['win_odds'],x['draw_odds'],x['loss_odds'],x['over_2.5goals_odds'],x['under_2.5goals_odds'],x['score'],x['away_score']), axis=1, result_type='expand')
outcome = outcome.rename(columns={0: "win_odds_pred",1:"draw_odds_pred",2:"loss_odds_pred",3:"over_odds_pred",4:"under_odds_pred",5:"pred",6:"pred_profit",7:"goals_pred",8:"goals_pred_profit",9:"sensible_profit",10:"sensible_profit_goals"})
X_calc[["win_odds_pred","draw_odds_pred","loss_odds_pred","over_odds_pred","under_odds_pred","pred","pred_profit","goals_pred","goals_pred_profit","sensible_profit","sensible_profit_goals"]] = outcome
X_calc

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing,expected,away_expected,away_score,win_odds_pred,draw_odds_pred,loss_odds_pred,over_odds_pred,under_odds_pred,pred,pred_profit,goals_pred,goals_pred_profit,sensible_profit,sensible_profit_goals
1736,7680,Newcastle Utd,Tottenham,2021,15:30,52214,2020-21,2,Karl Darlow,Ciaran Clark,Jamaal Lascelles,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Allan Saint-Maximin,Callum Wilson,Hugo Lloris,Eric Dier,Cristian Romero,Sergio Reguilón,Emerson,Tanguy Ndombele,Oliver Skipp,Pierre-Emile Hojbjerg,Harry Kane,Son Heung-Min,Lucas Moura,3.51,3.58,2.09,1.79,2.07,17,10,2021-10-17 15:30:00,2,1,2,9,7,4,1,0,9,5,3,1,2,13,9,4,2,0,9,6,1,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,7,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,23,1,1,1,1,1,1,1.141555,1.699308,3.0,3.925090,4.168144,2.015434,1.882931,2.175000,loss,10.9,over,7.9,10.9,7.9
1651,7684,Crystal Palace,Newcastle Utd,2021,14:00,24609,2020-21,1,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,James McArthur,Conor Gallagher,Christian Benteke,Odsonne Édouard,Michael Olise,Karl Darlow,Jamaal Lascelles,Ciaran Clark,Emil Krafth,Matt Ritchie,Javier Manquillo,Sean Longstaff,Ryan Fraser,Isaac Hayden,Allan Saint-Maximin,Callum Wilson,1.97,3.61,3.88,1.95,1.89,23,10,2021-10-23 14:00:00,2,0,3,11,5,2,1,2,11,8,2,1,4,13,10,3,2,2,15,12,1,14,12,8,18,41,66,-25,44,32.4,57.5,-25.0,-0.66,24,29.1,40.1,39,29,36,3,4,1.03,0.76,1.79,0.95,1.71,29.5,0.85,0.55,1.40,0.78,1.33,66,47,63,3,4,44,4,1.74,1.24,2.97,1.66,2.89,57.5,54.4,39.8,94.2,1.51,1.05,2.56,1.43,2.48,346,129,37.3,9.11,3.39,0.10,0.28,16.8,18,0.09,6.6,6.5,11,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27

In [51]:
print(X_calc['pred_profit'].sum())
print(X_calc['goals_pred_profit'].sum())
print(X_calc['sensible_profit'].sum())
print(X_calc['sensible_profit_goals'].sum())
print((X_calc['sensible_profit_goals'].sum() + X_calc['sensible_profit'].sum() + X_calc['goals_pred_profit'].sum() + X_calc['pred_profit'].sum()) /(len(X_calc)*40))

570.3000000000001
-1.8999999999999915
272.9
88.99999999999997
0.1435648148148148


In [52]:
# Then RandomForest
lasso_results = X_calc.copy(deep=True)
y_pred = rfr.predict(X_test)
X_calc = addPred(y_pred)


exp = X_calc.apply(lambda x: getAway(x['is_home'], x['index']), axis=1, result_type='expand')
exp = exp.rename(columns={0: "away_expected",1:"away_score"})
X_calc[["away_expected","away_score"]] = exp.astype(float)
X_calc = X_calc[X_calc['away_expected'] != -1]
outcome = X_calc.apply(lambda x: profit(x['expected'], x['away_expected'],x['win_odds'],x['draw_odds'],x['loss_odds'],x['over_2.5goals_odds'],x['under_2.5goals_odds'],x['score'],x['away_score']), axis=1, result_type='expand')
outcome = outcome.rename(columns={0: "win_odds_pred",1:"draw_odds_pred",2:"loss_odds_pred",3:"over_odds_pred",4:"under_odds_pred",5:"pred",6:"pred_profit",7:"goals_pred",8:"goals_pred_profit",9:"sensible_profit",10:"sensible_profit_goals"})
X_calc[["win_odds_pred","draw_odds_pred","loss_odds_pred","over_odds_pred","under_odds_pred","pred","pred_profit","goals_pred","goals_pred_profit","sensible_profit","sensible_profit_goals"]] = outcome
X_calc

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing,expected,away_expected,away_score,win_odds_pred,draw_odds_pred,loss_odds_pred,over_odds_pred,under_odds_pred,pred,pred_profit,goals_pred,goals_pred_profit,sensible_profit,sensible_profit_goals
1736,7680,Newcastle Utd,Tottenham,2021,15:30,52214,2020-21,2,Karl Darlow,Ciaran Clark,Jamaal Lascelles,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Allan Saint-Maximin,Callum Wilson,Hugo Lloris,Eric Dier,Cristian Romero,Sergio Reguilón,Emerson,Tanguy Ndombele,Oliver Skipp,Pierre-Emile Hojbjerg,Harry Kane,Son Heung-Min,Lucas Moura,3.51,3.58,2.09,1.79,2.07,17,10,2021-10-17 15:30:00,2,1,2,9,7,4,1,0,9,5,3,1,2,13,9,4,2,0,9,6,1,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,7,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,23,1,1,1,1,1,1,1.173506,1.706222,3.0,3.828983,4.182934,2.039559,1.852310,2.218927,loss,10.9,over,7.9,10.9,7.9
1651,7684,Crystal Palace,Newcastle Utd,2021,14:00,24609,2020-21,1,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,James McArthur,Conor Gallagher,Christian Benteke,Odsonne Édouard,Michael Olise,Karl Darlow,Jamaal Lascelles,Ciaran Clark,Emil Krafth,Matt Ritchie,Javier Manquillo,Sean Longstaff,Ryan Fraser,Isaac Hayden,Allan Saint-Maximin,Callum Wilson,1.97,3.61,3.88,1.95,1.89,23,10,2021-10-23 14:00:00,2,0,3,11,5,2,1,2,11,8,2,1,4,13,10,3,2,2,15,12,1,14,12,8,18,41,66,-25,44,32.4,57.5,-25.0,-0.66,24,29.1,40.1,39,29,36,3,4,1.03,0.76,1.79,0.95,1.71,29.5,0.85,0.55,1.40,0.78,1.33,66,47,63,3,4,44,4,1.74,1.24,2.97,1.66,2.89,57.5,54.4,39.8,94.2,1.51,1.05,2.56,1.43,2.48,346,129,37.3,9.11,3.39,0.10,0.28,16.8,18,0.09,6.6,6.5,11,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27

In [53]:
print(X_calc['pred_profit'].sum())
print(X_calc['goals_pred_profit'].sum())
print(X_calc['sensible_profit'].sum())
print(X_calc['sensible_profit_goals'].sum())
print((X_calc['sensible_profit_goals'].sum() + X_calc['sensible_profit'].sum() + X_calc['goals_pred_profit'].sum() + X_calc['pred_profit'].sum()) /(len(X_calc)*40))

612.9000000000001
-5.0
-0.0999999999999801
-31.400000000000013
0.08895061728395064


In [54]:
rfr_results = X_calc.copy(deep=True)
#Also we will try out Tensorflow DNN model despite it's poor MSE
y_pred = dnn_model.predict(X_test).flatten()
X_calc = addPred(y_pred)


exp = X_calc.apply(lambda x: getAway(x['is_home'], x['index']), axis=1, result_type='expand')
exp = exp.rename(columns={0: "away_expected",1:"away_score"})
X_calc[["away_expected","away_score"]] = exp.astype(float)
X_calc = X_calc[X_calc['away_expected'] != -1]
outcome = X_calc.apply(lambda x: profit(x['expected'], x['away_expected'],x['win_odds'],x['draw_odds'],x['loss_odds'],x['over_2.5goals_odds'],x['under_2.5goals_odds'],x['score'],x['away_score']), axis=1, result_type='expand')
outcome = outcome.rename(columns={0: "win_odds_pred",1:"draw_odds_pred",2:"loss_odds_pred",3:"over_odds_pred",4:"under_odds_pred",5:"pred",6:"pred_profit",7:"goals_pred",8:"goals_pred_profit",9:"sensible_profit",10:"sensible_profit_goals"})
X_calc[["win_odds_pred","draw_odds_pred","loss_odds_pred","over_odds_pred","under_odds_pred","pred","pred_profit","goals_pred","goals_pred_profit","sensible_profit","sensible_profit_goals"]] = outcome
X_calc

,index,team,opponent,year,time,attendance,league,score,starting_1,starting_2,starting_3,starting_4,starting_5,starting_6,starting_7,starting_8,starting_9,starting_10,starting_11,opp_starting_1,opp_starting_2,opp_starting_3,opp_starting_4,opp_starting_5,opp_starting_6,opp_starting_7,opp_starting_8,opp_starting_9,opp_starting_10,opp_starting_11,win_odds,draw_odds,loss_odds,over_2.5goals_odds,under_2.5goals_odds,day,month,datetime,wins_in_last5,losses_in_last5,ties_in_last5,goals_in_last5,goals_against_in_last5,opp_wins_in_last5,opp_losses_in_last5,opp_ties_in_last5,opp_goals_in_last5,opp_goals_against_in_last5,wins_in_last10,losses_in_last10,ties_in_last10,goals_in_last10,goals_against_in_last10,opp_wins_in_last10,opp_losses_in_last10,opp_ties_in_last10,opp_goals_in_last10,opp_goals_against_in_last10,is_home,Rk,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,# Pl,Age,Poss,Gls,Ast,G-PK,PK,PKatt,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,npxG,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Top Scorer Goals,opp_Rk,opp_W,opp_D,opp_L,opp_GF,opp_GA,opp_GD,opp_Pts,opp_xG,opp_xGA,opp_xGD,opp_xGD/90,opp_# Pl,opp_Age,opp_Poss,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_npxG,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Gls,opp_Ast,opp_G-PK,opp_PK,opp_PKatt,opp_CrdY,opp_CrdR,opp_Gls.1,opp_Ast.1,opp_G+A,opp_G-PK.1,opp_G+A-PK,opp_xG,opp_npxG,opp_xA,opp_npxG+xA,opp_xG.1,opp_xA.1,opp_xG+xA,opp_npxG.1,opp_npxG+xA.1,opp_Sh,opp_SoT,opp_SoT%,opp_Sh/90,opp_SoT/90,opp_G/Sh,opp_G/SoT,opp_Dist,opp_FK,opp_npxG/Sh,opp_G-xG,opp_np:G-xG,opp_Top Scorer Goals,Top Scorer playing,opp_Top Scorer playing,Best Player playing,opp_Best Player playing,Goalkeeper playing,opp_Goalkeeper playing,expected,away_expected,away_score,win_odds_pred,draw_odds_pred,loss_odds_pred,over_odds_pred,under_odds_pred,pred,pred_profit,goals_pred,goals_pred_profit,sensible_profit,sensible_profit_goals
1736,7680,Newcastle Utd,Tottenham,2021,15:30,52214,2020-21,2,Karl Darlow,Ciaran Clark,Jamaal Lascelles,Isaac Hayden,Matt Ritchie,Javier Manquillo,Sean Longstaff,Joe Willock,Joelinton,Allan Saint-Maximin,Callum Wilson,Hugo Lloris,Eric Dier,Cristian Romero,Sergio Reguilón,Emerson,Tanguy Ndombele,Oliver Skipp,Pierre-Emile Hojbjerg,Harry Kane,Son Heung-Min,Lucas Moura,3.51,3.58,2.09,1.79,2.07,17,10,2021-10-17 15:30:00,2,1,2,9,7,4,1,0,9,5,3,1,2,13,9,4,2,0,9,6,1,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,27,27.1,38.2,44,26,38,6,7,1.16,0.68,1.84,1.00,1.68,35.5,1.08,0.69,1.77,0.93,1.62,59,41,55,4,5,60,3,1.55,1.08,2.63,1.45,2.53,54.0,50.2,37.9,88.1,1.42,1.00,2.42,1.32,2.32,387,130,33.6,10.18,3.42,0.10,0.29,17.5,24,0.09,3.0,2.5,12,7,18,8,12,68,45,23,62,54.5,49.5,5.0,0.13,24,27.2,51.7,66,50,61,5,5,1.74,1.32,3.05,1.61,2.92,50.7,1.43,0.93,2.37,1.33,2.27,42,24,32,10,10,77,1,1.11,0.63,1.74,0.84,1.47,49.5,41.9,33.0,75.0,1.30,0.87,2.17,1.10,1.97,442,166,37.6,11.63,4.37,0.14,0.37,17.2,35,0.12,11.5,10.3,23,1,1,1,1,1,1,2.625754,1.164533,3.0,1.573956,6.014996,6.838621,1.476337,3.697279,win,-10.0,over,7.9,-10.0,7.9
1651,7684,Crystal Palace,Newcastle Utd,2021,14:00,24609,2020-21,1,Vicente Guaita,Marc Guehi,Joachim Andersen,Tyrick Mitchell,Joel Ward,Luka Milivojevic,James McArthur,Conor Gallagher,Christian Benteke,Odsonne Édouard,Michael Olise,Karl Darlow,Jamaal Lascelles,Ciaran Clark,Emil Krafth,Matt Ritchie,Javier Manquillo,Sean Longstaff,Ryan Fraser,Isaac Hayden,Allan Saint-Maximin,Callum Wilson,1.97,3.61,3.88,1.95,1.89,23,10,2021-10-23 14:00:00,2,0,3,11,5,2,1,2,11,8,2,1,4,13,10,3,2,2,15,12,1,14,12,8,18,41,66,-25,44,32.4,57.5,-25.0,-0.66,24,29.1,40.1,39,29,36,3,4,1.03,0.76,1.79,0.95,1.71,29.5,0.85,0.55,1.40,0.78,1.33,66,47,63,3,4,44,4,1.74,1.24,2.97,1.66,2.89,57.5,54.4,39.8,94.2,1.51,1.05,2.56,1.43,2.48,346,129,37.3,9.11,3.39,0.10,0.28,16.8,18,0.09,6.6,6.5,11,12,12,9,17,46,62,-16,45,41.0,54.0,-13.0,-0.34,2

In [57]:
print(X_calc['pred_profit'].sum())
print(X_calc['goals_pred_profit'].sum())
print(X_calc['sensible_profit'].sum())
print(X_calc['sensible_profit_goals'].sum())
print((X_calc['sensible_profit_goals'].sum() + X_calc['sensible_profit'].sum() + X_calc['goals_pred_profit'].sum() + X_calc['pred_profit'].sum()) /(len(X_calc)*40))

216.90000000000006
-368.70000000000005
31.700000000000017
-304.4
-0.06550925925925924


In [58]:
getParamsWithWeights(rfr.feature_importances_)

[[0.0008522359393735455, 'Goalkeeper playing'],
 [0.0008746235762640153, 'Gls.1'],
 [0.0009522993160185905, 'opp_xG.1'],
 [0.0009767781302478626, 'Gls'],
 [0.0009924092968810504, 'npxG/Sh'],
 [0.0012539617087114635, 'opp_Goalkeeper playing'],
 [0.0012611681389016686, 'opp_Gls'],
 [0.0013600235455860587, 'opp_xGA'],
 [0.0013889644006684376, 'opp_G-PK'],
 [0.0014220835408631917, 'opp_xG'],
 [0.0014250025735761582, 'opp_npxG+xA.1'],
 [0.0014327428011674953, 'xG.1'],
 [0.0014577821930814247, 'opp_G-PK.1'],
 [0.0014612247858438955, 'opp_Gls.1'],
 [0.0015114391779527015, 'G-PK'],
 [0.0015413519283006966, 'G-PK.1'],
 [0.0015855255604861171, 'opp_Ast'],
 [0.0015967859808252315, 'opp_Ast.1'],
 [0.0016503910522796307, 'opp_W'],
 [0.0016614196154067164, 'G/Sh'],
 [0.0017228707372370293, 'GA'],
 [0.0017684145030074266, 'opp_npxG.1'],
 [0.0018117290008605729, 'opp_PKatt'],
 [0.0018339099290024677, 'npxG+xA.1'],
 [0.001920877833723746, 'opp_Rk'],
 [0.001954300254502518, 'opp_Top Scorer playing'],
 [

In [59]:
getParamsWithWeights(en.coef_)

[[-0.006452798421396093, 'goals_against_in_last10'],
 [-0.005693336032532298, 'opp_Top Scorer Goals'],
 [-0.003126281030706138, 'opp_SoT'],
 [-0.0031215485694597533, 'opp_Gls'],
 [-0.0029226266651788545, 'opp_GA'],
 [-0.0028186629389110505, 'opp_CrdY'],
 [-0.00024057911223991426, 'CrdY'],
 [0.00023999263405848827, 'Sh'],
 [0.0003735143059195227, 'opp_Sh'],
 [0.0023732273518424526, 'G-xG'],
 [0.0027766500913285253, 'GD'],
 [0.003415337008799745, 'Ast'],
 [0.0037505705282317317, 'opp_FK'],
 [0.004179234949858262, 'SoT'],
 [0.006662502667583278, 'opp_npxG+xA'],
 [0.008150496672664353, 'Top Scorer Goals'],
 [0.012226892903838726, 'np:G-xG'],
 [0.022239937205323444, 'loss_odds']]

In [60]:
getParamsWithWeights(lasso.coef_)

[[-0.0029624003595367632, 'opp_SoT'],
 [-0.0007747514042056661, 'opp_Sh'],
 [0.002318712088411244, 'opp_npxG+xA'],
 [0.004211352352624075, 'GD'],
 [0.0059410367990244215, 'SoT']]